In [1]:
import json
import pandas as pd
import numpy as np
import os

from minio import Minio
from minio.error import S3Error

In [2]:
# COS_API_KEY_ID = os.environ.get("COS_API_KEY_ID", "none")
COS_ENDPOINT = os.environ.get("COS_ENDPOINT", "ibm-lh-lakehouse-minio-svc.cpd.svc.cluster.local:9000")
COS_ACCESS_KEY = os.environ.get("COS_ACCESS_KEY", "none")
COS_SECRET_KEY = os.environ.get("COS_SECRET_KEY", "none")



# print("key: " + COS_API_KEY_ID)
# if COS_API_KEY_ID == "none":
#     print("No COS API KEY!!")
#     #for name, value in os.environ.items():
#     #    print("{0}: {1}".format(name, value))

In [3]:
# ibm s3
# import ibm_boto3
# from ibm_botocore.client import Config, ClientError

In [4]:
# Parameters

description="Return all the ESG dimensions for a single portfolio ticker"

output="output.json"

input={"ticker": "ABNP"}

results={"Ticker": "ABNP",
         "ISIN": "OTOY0BBKWAA1",
         "Industry": "Homebuilding and construction supplies",
         "ESG_properties": [
             {"C02": 0.59},
             {"TotalWaste": 0.86}, 
             {"EnvProducts": None}, 
             {"EnvRND": 0.87}, 
             {"WaterUse": 0.77}, 
             {"EnergyUse": 0.61}, 
             {"PolicyHumanRights": 0.84}, 
             {"PolicyResponsibleMarketing": None}, 
             {"ProductQualityMonitoring": None}, 
             {"PolicyDataPrivacy": 0.45}, 
             {"WomenEmployees": 0.33}, 
             {"AvgTrainingHours": 0.97}, 
             {"TradeUnionRep": 0.48}, 
             {"LostDays": 0.51}]}

Test curl post:

    curl -X POST https://datamon.15nwk2ygtk89.eu-de.codeengine.appdomain.cloud/api/ESGFastTickerService -H 'Content-Type: application/json' -d '{"portfolio": "ABNF"}'

In [5]:
# Constants for IBM COS values
# COS_ENDPOINT = "https://s3.us-east.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
#COS_API_KEY_ID = # <api-key> eg "W00YixxxxxxxxxxMB-odB-2ySfTrFBIQQWanc--P3byk"
# COS_INSTANCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/b71ac2564ef0b98f1032d189795994dc:f7e4281d-a7f2-4923-8e6b-07f99ac93bbd::" # <service-instance-id> eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003xxxxxxxxxx1c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"

# Create client 
# s3_client = ibm_boto3.client("s3",
#     ibm_api_key_id=COS_API_KEY_ID,
#     ibm_service_instance_id=COS_INSTANCE_CRN,
#     config=Config(signature_version="oauth"),
#     endpoint_url=COS_ENDPOINT
# )

# COS_ENDPOINT='ibm-lh-lakehouse-minio-svc.cpd.svc.cluster.local:9000'

minio_client = Minio(
    endpoint=COS_ENDPOINT,
    secure=False,
    access_key=COS_ACCESS_KEY,
    secret_key=COS_SECRET_KEY
)


file_name = "provider_a/ESG-consolidated.csv"
bucket = "delivery"




print("Retrieving item from bucket: {0}, key: {1}".format(bucket, file_name))
try:
    minio_client.fget_object(bucket, file_name,file_name)
except S3Error as e:
    e


Retrieving item from bucket: delivery, key: provider_a/ESG-consolidated.csv


In [6]:
source = pd.read_csv(file_name)
source.head(5)

,Ticker,Isin,Industry,Co2,Totalwaste,Envproducts,Envrnd,Wateruse,Energyuse,Policyhumanrights,Policyresponsiblemarketing,Productqualitymonitoring,Policydataprivacy,Womenemployees,Avgtraininghours,Tradeunionrep,Lostdays
0,JSJC,YACEFSEQB3A4,Office equipment,0.95,NaN,NaN,NaN,0.78,0.89,0.22,NaN,NaN,0.19,0.53,0.18,0.54,0.22
1,JSKE,QL6UOE2EJCJ9,Water and related utilities,NaN,0.17,0.99,0.10,0.07,NaN,0.74,0.18,0.28,0.92,0.78,0.74,0.43,0.69
2,JSKK,EGJL1NEBFRT2,Freight and logistics services,0.18,0.85,NaN,0.16,0.75,0.37,NaN,0.53,0.81,0.55,0.64,0.25,0.62,0.91
3,JSMD,OM9BJUQENP52,Leisure products,0.70,NaN,0.02,0.45,0.57,NaN,0.13,NaN,0.33,0.79,0.30,0.25,0.51,0.18
4,JSN,FB6W2YINKWE5,Insurance,0.04,0.37,0.95,NaN,NaN,0.94,0.02,0.13,0.93,0.32,0.79,0.87,0.97,0.14


In [7]:
source = source[source['Ticker'] == input['ticker']]
source

,Ticker,Isin,Industry,Co2,Totalwaste,Envproducts,Envrnd,Wateruse,Energyuse,Policyhumanrights,Policyresponsiblemarketing,Productqualitymonitoring,Policydataprivacy,Womenemployees,Avgtraininghours,Tradeunionrep,Lostdays
6482,ABNP,OTOY0BBKWAA1,Homebuilding and construction supplies,0.95,0.86,0.44,0.87,0.77,0.61,0.84,NaN,0.78,0.45,0.33,0.97,0.48,0.51


In [8]:
output_dict={}
output_dict['Ticker'] = source['Ticker'].values[0]
output_dict['ISIN'] = source['Isin'].values[0]
output_dict['Industry'] = source['Industry'].values[0]
output_dict

{'Ticker': 'ABNP',
 'ISIN': 'OTOY0BBKWAA1',
 'Industry': 'Homebuilding and construction supplies'}

In [9]:
cols = source.columns.values
cols = cols[3:]
cols

array(['Co2', 'Totalwaste', 'Envproducts', 'Envrnd', 'Wateruse',
       'Energyuse', 'Policyhumanrights', 'Policyresponsiblemarketing',
       'Productqualitymonitoring', 'Policydataprivacy', 'Womenemployees',
       'Avgtraininghours', 'Tradeunionrep', 'Lostdays'], dtype=object)

In [10]:
property_list=[]
for dimension in cols:
    obj_dict={}
    obj_dict[dimension] = source[dimension].values[0]
    property_list.append(obj_dict)
property_list

[{'Co2': 0.95},
 {'Totalwaste': 0.86},
 {'Envproducts': 0.44},
 {'Envrnd': 0.87},
 {'Wateruse': 0.77},
 {'Energyuse': 0.61},
 {'Policyhumanrights': 0.84},
 {'Policyresponsiblemarketing': nan},
 {'Productqualitymonitoring': 0.78},
 {'Policydataprivacy': 0.45},
 {'Womenemployees': 0.33},
 {'Avgtraininghours': 0.97},
 {'Tradeunionrep': 0.48},
 {'Lostdays': 0.51}]

In [11]:
output_dict["ESG_properties"] = property_list
output_dict

{'Ticker': 'ABNP',
 'ISIN': 'OTOY0BBKWAA1',
 'Industry': 'Homebuilding and construction supplies',
 'ESG_properties': [{'Co2': 0.95},
  {'Totalwaste': 0.86},
  {'Envproducts': 0.44},
  {'Envrnd': 0.87},
  {'Wateruse': 0.77},
  {'Energyuse': 0.61},
  {'Policyhumanrights': 0.84},
  {'Policyresponsiblemarketing': nan},
  {'Productqualitymonitoring': 0.78},
  {'Policydataprivacy': 0.45},
  {'Womenemployees': 0.33},
  {'Avgtraininghours': 0.97},
  {'Tradeunionrep': 0.48},
  {'Lostdays': 0.51}]}

In [12]:
jsonString = json.dumps(output_dict)
jsonString

'{"Ticker": "ABNP", "ISIN": "OTOY0BBKWAA1", "Industry": "Homebuilding and construction supplies", "ESG_properties": [{"Co2": 0.95}, {"Totalwaste": 0.86}, {"Envproducts": 0.44}, {"Envrnd": 0.87}, {"Wateruse": 0.77}, {"Energyuse": 0.61}, {"Policyhumanrights": 0.84}, {"Policyresponsiblemarketing": NaN}, {"Productqualitymonitoring": 0.78}, {"Policydataprivacy": 0.45}, {"Womenemployees": 0.33}, {"Avgtraininghours": 0.97}, {"Tradeunionrep": 0.48}, {"Lostdays": 0.51}]}'

In [13]:
 # Writing to sample.json
with open(output, "w") as outfile:
    outfile.write(jsonString)